In [1]:
import numpy as np
import tls_client
import pandas as pd
import webbrowser
import requests
import re
import plotly.express as px
import seaborn as sns
from utils import merge_books, pp_delta, ud_delta
import json

pd.set_option('display.max_columns', None)

In [2]:
# -- read in scraped data from RotoWorld

pp = pd.read_csv(r"C:\Users\mikej\Downloads\rw-prizepicks-predictions-2024-01-31.csv")         #prize picks lines
ud = pd.read_csv(r"C:\Users\mikej\Downloads\rw-underdog-predictions-2024-01-31.csv")           #underdog lines

spreads = pd.read_excel(r"C:\Users\mikej\Downloads\nfl-odds-rotowire (8).xlsx",header=1)           #game spreads and totals
spreads = (spreads
           [['Team','Spread','Over-Under']]
           .rename(columns = {'Team':'team','Spread':'spread','Over-Under':'over_under'}))

books_merged = merge_books(pp,ud)
books_merged.to_csv('data/books_merged.csv',index=False)
books_merged.sample(3)

,player_display_name,position,team,opponent,market,pp_line,ud_line
4,Travis Kelce,TE,KC,SF,Receiving Yards,72.5,69.50
92,Deebo Samuel,WR,SF,@KC,Targets,NaN,6.50
52,Deebo Samuel,WR,SF,@KC,Fantasy Score,14.0,12.05


In [3]:
# ------- REQUEST DATA FROM NFL API -------

import nfl_data_py as nfl

years = [2023,2024]
nfl_df = nfl.import_weekly_data(years,downcast=True)

Downcasting floats.


In [4]:
##  MERGE SPORTSBOOKS AND NFL PLAYER DATA

books_merged = pd.read_csv(r"C:\Users\mikej\Desktop\mean-median\data\books_merged.csv")
all_merged = books_merged.merge(nfl_df, how='left', on='player_display_name')
print(f"{all_merged.player_display_name.nunique()} players with available spreads")

29 players with available spreads


In [6]:
all_merged

,player_display_name,position_x,team,opponent,market,pp_line,ud_line,player_id,player_name,position_y,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,Patrick Mahomes,QB,KC,SF,Passing Yards,0.5,261.50,00-0033873,P.Mahomes,QB,QB,https://static.www.nfl.com/image/private/f_aut...,KC,2023.0,1.0,REG,DET,21.0,39.0,226.0,2.0,1.0,0.0,-0.0,0.0,0.0,354.0,113.0,10.0,-6.453613,0.0,0.638418,0.024097,6.0,45.0,0.0,0.0,0.0,3.0,2.924736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,19.540001,19.540001
1,Patrick Mahomes,QB,KC,SF,Passing Yards,0.5,261.50,00-0033873,P.Mahomes,QB,QB,https://static.www.nfl.com/image/private/f_aut...,KC,2023.0,2.0,REG,JAX,29.0,41.0,305.0,2.0,1.0,1.0,7.0,0.0,0.0,274.0,179.0,12.0,10.558776,0.0,1.113139,0.156275,7.0,30.0,0.0,1.0,0.0,3.0,-2.991675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,21.200001,21.200001
2,Patrick Mahomes,QB,KC,SF,Passing Yards,0.5,261.50,00-0033873,P.Mahomes,QB,QB,https://static.www.nfl.com/image/private/f_aut...,KC,2023.0,3.0,REG,CHI,24.0,33.0,272.0,3.0,0.0,0.0,-0.0,0.0,0.0,223.0,117.0,14.0,20.917330,0.0,1.219731,0.285791,3.0,28.0,0.0,0.0,0.0,2.0,1.337047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,25.680000,25.680000
3,Patrick Mahomes,QB,KC,SF,Passing Yards,0.5,261.50,00-0033873,P.Mahomes,QB,QB,https://static.www.nfl.com/image/private/f_aut...,KC,2023.0,4.0,REG,NYJ,18.0,30.0,203.0,1.0,2.0,1.0,6.0,0.0,0.0,195.0,139.0,9.0,0.632651,0.0,1.041026,0.092659,7.0,51.0,0.0,1.0,0.0,4.0,2.218627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,13.220000,13.220000
4,Patrick Mahomes,QB,KC,SF,Passing Yards,0.5,261.50,00-0033873,P.Mahomes,QB,QB,https://static.www.nfl.com/image/private/f_aut...,KC,2023.0,5.0,REG,MIN,31.0,41.0,281.0,2.0,0.0,2.0,15.0,0.0,0.0,241.0,148.0,15.0,14.760243,0.0,1.165975,0.252179,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,19.240000,19.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,Justin Watson,WR,KC,SF,Fantasy Score,NaN,3.35,00-0034386,J.Watson,WR,WR,https://static.www.nfl.com/image/private/f_aut...,KC,2024.0,4.0,REG,LAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,2.0,3.0,27.0,0.0,0.0,0.0,29.0,4.0,1.0,2.447961,0.0,0.931035,0.107143,0.154255,0.268693,0.0,2.700000,4.700000
1771,Justin Watson,WR,KC,SF,Fantasy Score,NaN,3.35,00-0034386,J.Watson,WR,WR,https://static.www.nfl.com/image/private/f_aut...,KC,2024.0,7.0,REG,SF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,8.0,0.0,0.0,0.0,5.0,3.0,0.0,0.715891,0.0,1.600000,0.040000,0.023148,0.076204,0.0,0.800000,1.800000
1772,Harrison Butker,K,KC,SF,Kicking Points,NaN,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1773,Fred Warner,LB,SF,@KC,Solo Tackles,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [8]:
###  -------------- TO RERUN START HERE!!!! ---------

# add this weeks opponent
df = all_merged.copy().reset_index(drop=True)
df['opponent_team'].fillna('None',inplace=True)

df_filtered_by_stat = df[(df.market == 'Rushing Yards') | (df.market == 'Passing Yards') | (df.market == 'Receiving Yards')]

filtered_by_needed_columns = df_filtered_by_stat[['player_display_name', 'position_x', 'team', 'opponent',
       'market', 'pp_line','ud_line', 'player_id', 'player_name','season', 'week','completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks','carries', 'rushing_yards', 'rushing_tds','receptions', 'targets', 'receiving_yards',
       'receiving_tds','receiving_air_yards','target_share','wopr',
       'fantasy_points','opponent_team']].rename(columns={'position_x':'position'}).reset_index(drop=True)

#    cleanup wording to match nfl data
stats_dict = {'Rushing Yards':'rushing_yards', 'Passing Yards':'passing_yards','Receiving Yards':'receiving_yards'}
filtered_by_needed_columns['market'] = filtered_by_needed_columns['market'].map(stats_dict)
df = filtered_by_needed_columns.copy()

# --- add 'delta' columns using functions from 'utils' module
df['pp_delta'] = df.apply(pp_delta, axis=1)
df['ud_delta'] = df.apply(ud_delta, axis=1)

# add long team name
team_full = ['Cardinals','Falcons','Ravens','Bills','Panthers','Bears','Bengals','Browns','Cowboys','Broncos','Lions','Packers','Texans','Colts','Jaguars','Chiefs','Raiders','Chargers','Rams','Dolphins','Vikings','Patriots','Saints','Giants','Jets','Eagles','Steelers','49ers','Seahawks','Buccaneers','Titans','Commanders']
team_abr = ['ARI','ATL','BAL','BUF','CAR','CHI','CIN','CLE','DAL','DEN','DET','GB','HOU','IND','JAX','KC','LV','LAC','LAR','MIA','MIN','NE','NO','NYG','NYJ','PHI','PIT','SF','SEA','TB','TEN','WAS']
zipped = list(zip(team_abr,team_full))
team_dict = dict(zipped)
df['team_long'] = df['team'].map(team_dict)

# add game spreads and totals
zipped = list(zip(spreads.team,spreads.spread))
spread_dict = dict(zipped)
df['spread'] = df['team_long'].map(spread_dict)

zipped = list(zip(spreads.team,spreads.over_under))
ou_dict = dict(zipped)
df['over_under'] = df['team_long'].map(ou_dict)

print(f"DataFrame shape: {df.shape}")
print("")
df.sample(3)

DataFrame shape: (485, 34)



,player_display_name,position,team,opponent,market,pp_line,ud_line,player_id,player_name,season,week,completions,attempts,passing_yards,passing_tds,interceptions,sacks,carries,rushing_yards,rushing_tds,receptions,targets,receiving_yards,receiving_tds,receiving_air_yards,target_share,wopr,fantasy_points,opponent_team,pp_delta,ud_delta,team_long,spread,over_under
169,Deebo Samuel,WR,SF,@KC,receiving_yards,57.5,55.5,00-0035719,D.Samuel,2023.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,0.0,3.0,11.0,33.0,0.0,123.0,0.297297,0.726402,4.100000,KC,-24.5,-22.5,49ers,-2,47
253,Brock Purdy,QB,SF,@KC,passing_yards,247.5,243.5,00-0037834,B.Purdy,2023.0,11.0,21.0,25.0,333.0,3.0,0.0,4.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,26.719999,TB,85.5,89.5,49ers,-2,47
288,Christian McCaffrey,RB,SF,@KC,rushing_yards,90.5,92.5,00-0033280,C.McCaffrey,2023.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,80.0,0.0,8.0,8.0,80.0,1.0,-2.0,0.216216,0.319764,20.000000,KC,-10.5,-12.5,49ers,-2,47
